# SE446 - Week 3A: Introduction to MapReduce

## 📚 Learning Objectives

By the end of this notebook, you will be able to:
1. Understand the MapReduce programming model
2. Trace data flow through Map → Shuffle → Reduce phases
3. Implement simple MapReduce operations in Python
4. Apply word count as the canonical example

---

## 1. Why MapReduce?

### The Problem with Traditional Processing

Imagine you have **10 TB** of web server logs and need to count page views.

**Traditional approach:**
- Load all data into memory ❌ (doesn't fit)
- Process sequentially ❌ (takes forever)
- Single point of failure ❌

**MapReduce approach:**
- Distribute data across nodes ✅
- Process in parallel ✅
- Automatic fault tolerance ✅

### The Core Idea: Divide and Conquer

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   Data 1    │     │   Data 2    │     │   Data 3    │
└──────┬──────┘     └──────┬──────┘     └──────┬──────┘
       │                   │                   │
       ▼                   ▼                   ▼
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   MAP 1     │     │   MAP 2     │     │   MAP 3     │
│ (parallel)  │     │ (parallel)  │     │ (parallel)  │
└──────┬──────┘     └──────┬──────┘     └──────┬──────┘
       │                   │                   │
       └───────────────────┼───────────────────┘
                           │
                           ▼
                  ┌─────────────────┐
                  │  SHUFFLE & SORT │
                  │  (group by key) │
                  └────────┬────────┘
                           │
                           ▼
                  ┌─────────────────┐
                  │     REDUCE      │
                  │   (aggregate)   │
                  └────────┬────────┘
                           │
                           ▼
                       RESULT
```

## 2. MapReduce Phases Explained

### 2.1 The Map Phase

**Purpose**: Transform each input record into (key, value) pairs

```python
map(key, value) → list[(key', value')]
```

- Processes **one record at a time**
- Can emit **zero, one, or multiple** key-value pairs
- Runs **in parallel** across the cluster

In [ ]:
# Example: Word Count Mapper

def word_count_mapper(line):
    """
    Input: A line of text (string)
    Output: List of (word, 1) tuples
    """
    results = []
    for word in line.split():
        results.append((word.lower(), 1))
    return results

# Test the mapper
test_line = "Hello World Hello Big Data"
mapped = word_count_mapper(test_line)
print("Input:", test_line)
print("Mapper output:", mapped)

### 2.2 The Shuffle Phase

**Purpose**: Group all values by their key

This phase is **automatic** - you don't write code for it!

```
Before Shuffle:                    After Shuffle:
(hello, 1)                         hello → [1, 1]
(world, 1)                         world → [1]
(hello, 1)                         big   → [1]
(big, 1)                           data  → [1]
(data, 1)
```

In [ ]:
# Simulating the Shuffle Phase
from collections import defaultdict

def shuffle(mapped_pairs):
    """
    Group all values by their key
    Input: List of (key, value) tuples
    Output: Dictionary {key: [values]}
    """
    grouped = defaultdict(list)
    for key, value in mapped_pairs:
        grouped[key].append(value)
    return dict(grouped)

# Test shuffle
shuffled = shuffle(mapped)
print("After shuffle:")
for key, values in shuffled.items():
    print(f"  {key} → {values}")

### 2.3 The Reduce Phase

**Purpose**: Aggregate values for each key into a final result

```python
reduce(key, list[values]) → (key, result)
```

- Receives **all values for one key**
- Produces **one output per key** (usually)

In [ ]:
# Example: Word Count Reducer

def word_count_reducer(key, values):
    """
    Input: word (string), list of counts [1, 1, 1, ...]
    Output: (word, total_count)
    """
    return (key, sum(values))

# Test the reducer
results = []
for word, counts in shuffled.items():
    result = word_count_reducer(word, counts)
    results.append(result)
    print(f"Reduce({word}, {counts}) → {result}")

print("\nFinal results:", dict(results))

## 3. Complete MapReduce Framework

Let's build a simple but complete MapReduce framework in Python:

In [ ]:
from collections import defaultdict

def map_reduce(data, mapper, reducer):
    """
    A simple MapReduce implementation.
    
    Parameters:
    - data: Input data (list of records)
    - mapper: Function that takes a record and returns list of (key, value)
    - reducer: Function that takes (key, [values]) and returns (key, result)
    
    Returns:
    - List of (key, result) tuples
    """
    # ========== MAP PHASE ==========
    mapped = []
    for record in data:
        pairs = mapper(record)
        if pairs:  # Handle mappers that return None
            if isinstance(pairs, list):
                mapped.extend(pairs)
            else:
                mapped.append(pairs)
    
    print(f"📤 Map phase: {len(data)} records → {len(mapped)} pairs")
    
    # ========== SHUFFLE PHASE ==========
    shuffled = defaultdict(list)
    for key, value in mapped:
        shuffled[key].append(value)
    
    print(f"🔀 Shuffle phase: {len(mapped)} pairs → {len(shuffled)} groups")
    
    # ========== REDUCE PHASE ==========
    results = []
    for key, values in shuffled.items():
        result = reducer(key, values)
        if result:  # Handle reducers that return None
            results.append(result)
    
    print(f"📥 Reduce phase: {len(shuffled)} groups → {len(results)} results")
    
    return results

## 4. Word Count - The Complete Example

In [ ]:
# Sample documents
documents = [
    "Hello World Hello",
    "World of Big Data",
    "Big Data is the Future",
    "Hello Future World"
]

# Define mapper and reducer
def mapper(line):
    """Split line into words, emit (word, 1) for each"""
    return [(word.lower(), 1) for word in line.split()]

def reducer(word, counts):
    """Sum all counts for a word"""
    return (word, sum(counts))

# Run MapReduce
print("=" * 50)
print("WORD COUNT WITH MAPREDUCE")
print("=" * 50)

word_counts = map_reduce(documents, mapper, reducer)

print("\n📊 Results:")
for word, count in sorted(word_counts, key=lambda x: x[1], reverse=True):
    print(f"  {word}: {count}")

## 5. 🎯 Exercise: Tracing Data Flow

For the input: `["A B A", "B C B"]`

Trace through each phase manually:

**Step 1: Map Phase**
- Line 1 "A B A" → ?
- Line 2 "B C B" → ?

**Step 2: Shuffle Phase**
- A → ?
- B → ?
- C → ?

**Step 3: Reduce Phase**
- A, [?] → ?
- B, [?] → ?
- C, [?] → ?

In [ ]:
# Verify your answer
test_input = ["A B A", "B C B"]
test_results = map_reduce(test_input, mapper, reducer)
print("\nFinal answer:", dict(test_results))

## 6. Key Concepts Summary

### MapReduce Components

| Component | Input | Output | Your Job |
|-----------|-------|--------|----------|
| **Mapper** | One record | (key, value) pairs | Write this function |
| **Shuffle** | All map outputs | Grouped by key | Automatic |
| **Reducer** | Key + all values | Final result | Write this function |

### The Golden Rule

> **"Think in key-value pairs!"**
>
> When designing MapReduce:
> 1. What should the **key** be? (What do you want to group by?)
> 2. What should the **value** be? (What do you want to aggregate?)

## 7. 🏠 Pre-Class Preparation for Session 3B

1. **Watch**: Python MapReduce Tutorial video
2. **Think**: How would you count crimes by type using MapReduce?
3. **Review**: Chicago crime dataset columns

In Session 3B, we'll apply MapReduce to real crime data analysis!